In [10]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import WazeRouteCalculator
import datetime

## Pulling Job Data

In [11]:
project_path = "/Users/jjespinoza/GoogleDrive/2_projects/2_itsfriday/"
now = datetime.datetime.now()
now.year
now.month
now.day
date = str(now.year) + str(now.month) + str(now.day) + "_" + str(now.hour) + str(now.minute)
date


'2018117_1235'

In [17]:
max_results_per_city = 10
city_set = ['Los+Angeles']
#city_set = ['Los+Angeles', 'Los+Angeles+County', 'Long+Beach', 'Downey', 'Commerce']
title_set = ['data+scientist']
#title_set = ['data+scientist', 'senior+data+scientist', 'director+data+science', 'director+analytics', 'vice+president+analytics', 'vice+president+data+science']
columns = ['city', 'job_title', 'company_name', 'location', 'summary', 'salary']
sample_df = pd.DataFrame(columns = columns)


In [18]:
#scraping code:
for title in title_set:
    for city in city_set:
        for start in range(0, max_results_per_city, 10):
            page = requests.get('http://www.indeed.com/jobs?q='+ str(title) +'+%2420%2C000&l=' + str(city) + '&start=' + str(start))
            time.sleep(1)  #ensuring at least 1 second between page grabs
            soup = BeautifulSoup(page.text, 'lxml')
            for div in soup.find_all(name='div', attrs={'class':'row'}): 
                #specifying row num for index of job posting in dataframe
                num = (len(sample_df) + 1) 
                #creating an empty list to hold the data for each posting
                job_post = [] 
                #append city name
                job_post.append(city) 
                #grabbing job title
                for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                    job_post.append(a['title']) 
                #grabbing company name
                company = div.find_all(name='span', attrs={'class':'company'}) 
                if len(company) > 0: 
                    for b in company:
                        job_post.append(b.text.strip()) 
                else: 
                    sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                    for span in sec_try:
                        job_post.append(span.text) 
                #grabbing location name
                c = div.findAll('span', attrs={'class': 'location'}) 
                for span in c: 
                    job_post.append(span.text) 
                #grabbing summary text
                d = div.findAll('span', attrs={'class': 'summary'}) 
                for span in d:
                    job_post.append(span.text.strip()) 
                #grabbing salary
                try:
                    job_post.append(div.find('nobr').text) 
                except:
                    try:
                        div_two = div.find(name='div', attrs={'class':'sjcl'}) 
                        div_three = div_two.find('div') 
                        job_post.append(div_three.text.strip())
                    except:
                        job_post.append('Nothing_found') 
                #appending list of job post info to dataframe at index num
                sample_df.loc[num] = job_post




In [19]:
del sample_df['city']

In [20]:
sample_df = sample_df.drop_duplicates()

In [21]:
sample_df

,job_title,company_name,location,summary,salary
1,Senior Data Scientist,Restoration Media,"Irvine, CA 92614",The Senior Data Scientist will leverage the bu...,Nothing_found
2,Data Scientist I,Auto Club of Southern California,"Costa Mesa, CA 92626",A Data Scientist is expected to understand the...,Nothing_found
3,Envionmental Scientist II,CWE,"Fullerton, CA 92831","Data management and analysis (compilation, man...",Nothing_found
4,"Associate, Data Scientist/Statistics - Financi...",KPMG,"Los Angeles, CA 90071","Machine learning, data visualization, statisti...",Nothing_found
5,Artificial Intelligence & Data Scientist,Twentieth Century Fox,"Los Angeles, CA",1-3+ years of experience as data scientist. Ex...,Nothing_found
6,Data Scientist / Data Analyst,Hyperloop One,"Los Angeles, CA",The Data Scientist / Data Analyst will be part...,Nothing_found
7,"Research Scientist, Machine Learning / Artific...",Disney Parks & Resorts,"Glendale, CA",Duties of a Research Scientist include:. Resea...,Nothing_found
8,Data Scientist - Tech Start Up,Next Trucking,"Los Angeles, CA",We are looking for a motivated Data Scientist ...,Nothing_found
9,Data Scientist – Machine Learning,Internet Brands,"Los Angeles, CA",Is currently seeking a Data Scientist - Machin...,Nothing_found
10,Associate Machine Learning Engineer,eharmony,"Los Angeles, CA",Create data pipelines to establish datasets fo...,Nothing_found


## Calculating Distance

In [22]:
sample_df["clean_address"] = sample_df["company_name"] + ' ' + sample_df["location"]

In [23]:
commutes = []

for address in sample_df["clean_address"]:
    from_address = "10757 Longworth Ave Santa Fe Springs CA 90670"
    region = 'US'
    try:
        to_address = address
        route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region )
        commutes.append(route.calc_route_info(real_time=False))
    except:
        commutes.append((0,0))
    
    
        
    
        

From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Restoration Media Irvine, CA 92614
Time 33.43 minutes, distance 47.22 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Auto Club of Southern California Costa Mesa, CA 92626
Time 28.13 minutes, distance 40.59 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: CWE Fullerton, CA 92831
Time 26.07 minutes, distance 24.81 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: KPMG Los Angeles, CA 90071
Time 44.10 minutes, distance 43.50 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Twentieth Century Fox Los Angeles, CA
Time 42.35 minutes, distance 55.86 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Hyperloop One Los Angeles, CA
Time 20.42 minutes, distance 19.29 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Disney Parks & Resorts Glendale, CA
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Next Trucking Los Angeles, CA
Time 44.10 minutes, distance

In [25]:
df = pd.DataFrame(commutes) 
df.columns = ['commute', 'distance_km']
df.index = range(1,len(df)+1)
sample_df = sample_df.join(df, lsuffix='_caller', rsuffix='_other')
sample_df


,job_title,company_name,location,summary,salary,clean_address,commute_caller,distance_km_caller,commute_other,distance_km_other
1,Senior Data Scientist,Restoration Media,"Irvine, CA 92614",The Senior Data Scientist will leverage the bu...,Nothing_found,"Restoration Media Irvine, CA 92614",33.433333,47.222,33.433333,47.222
2,Data Scientist I,Auto Club of Southern California,"Costa Mesa, CA 92626",A Data Scientist is expected to understand the...,Nothing_found,"Auto Club of Southern California Costa Mesa, C...",28.133333,40.595,28.133333,40.595
3,Envionmental Scientist II,CWE,"Fullerton, CA 92831","Data management and analysis (compilation, man...",Nothing_found,"CWE Fullerton, CA 92831",26.066667,24.813,26.066667,24.813
4,"Associate, Data Scientist/Statistics - Financi...",KPMG,"Los Angeles, CA 90071","Machine learning, data visualization, statisti...",Nothing_found,"KPMG Los Angeles, CA 90071",44.100000,43.500,44.100000,43.500
5,Artificial Intelligence & Data Scientist,Twentieth Century Fox,"Los Angeles, CA",1-3+ years of experience as data scientist. Ex...,Nothing_found,"Twentieth Century Fox Los Angeles, CA",42.350000,55.857,42.350000,55.857
6,Data Scientist / Data Analyst,Hyperloop One,"Los Angeles, CA",The Data Scientist / Data Analyst will be part...,Nothing_found,"Hyperloop One Los Angeles, CA",20.416667,19.290,20.416667,19.290
7,"Research Scientist, Machine Learning / Artific...",Disney Parks & Resorts,"Glendale, CA",Duties of a Research Scientist include:. Resea...,Nothing_found,"Disney Parks & Resorts Glendale, CA",0.000000,0.000,0.000000,0.000
8,Data Scientist - Tech Start Up,Next Trucking,"Los Angeles, CA",We are looking for a motivated Data Scientist ...,Nothing_found,"Next Trucking Los Angeles, CA",44.100000,43.500,44.100000,43.500
9,Data Scientist – Machine Learning,Internet Brands,"Los Angeles, CA",Is currently seeking a Data Scientist - Machin...,Nothing_found,"Internet Brands Los Angeles, CA",44.800000,62.167,44.800000,62.167
10,Associate Machine Learning Engineer,eharmony,"Los Angeles, CA",Create data pipelines to establish datasets fo...,Nothing_found,"eharmony Los Angeles, CA",37.233333,43.162,37.233333,43.162


In [66]:
sample_df.to_csv(project_path + "2_data/1_raw/scraper_indeed_jobs_" + date + ".csv", encoding='utf-8')